In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import string
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
df_fake=pd.read_csv("Fake.csv")
df_true=pd.read_csv("True.csv")

In [21]:
df_fake["class"] = 0
df_true["class"] = 1

In [22]:
df_marge = pd.concat([df_fake, df_true], axis =0 )

In [23]:
df = df_marge.drop(["title", "subject","date"], axis = 1)

In [24]:
df = df.sample(frac = 1)

In [25]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [26]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [27]:
df["text"] = df["text"].apply(wordopt)

In [28]:
x = df["text"]
y = df["class"]

In [29]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [30]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

# LOGISTIC REGRESSION

In [31]:
from sklearn.linear_model import LogisticRegression

In [32]:
LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [33]:
pred_lr=LR.predict(xv_test)

In [34]:
sc=LR.score(xv_test, y_test)
print("Test score: {0:.2f} %".format(100 * sc))
print(classification_report(y_test, pred_lr))

Test score: 98.67 %
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5887
           1       0.98      0.99      0.99      5338

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [35]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)

    return print("\n\nLR Prediction: {} ".format(output_lable(pred_LR[0]),))


In [36]:
news = str(input())
manual_testing(news)

When life in her country became too dangerous, Natalia* took her children and fled. She managed to cross into Mexico via the southern border, but once she arrived in the country, she had no means of providing for herself and her family.   Her story is similar to those of many other desperate people who head to Mexico, increasingly seen as a country of transit and asylum: In 2014, 2,100 people arrived in the country to request refugee status; five years later, in 2019, that had risen to more than 70,000.  The figures dropped in 2020, as travel restrictions imposed as a result of the COVID-19 pandemic slowed global migration but, between January and November 2021, the country received more than 123,000 asylum requests from people coming from Caribbean, Central American and South American countries (Haiti, Honduras, Cuba, El Salvador, Chile, Venezuela, Guatemala, Nicaragua, Brazil and Colombia).  Natalia and her children are now being supported by the UN refugee agency, (UNHCR), which has

In [37]:
import pickle
pickle.dump(LR,open('model.pkl', 'wb'))

In [38]:
# load the model from disk
loaded_model = pickle.load(open('model.pkl', 'rb'))
score = loaded_model.score(xv_test, y_test)
print("Test score: {0:.2f} %".format(100 * score))

def model_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR1 = loaded_model.predict(new_xv_test)

    return print("\n\nModel Prediction: {} ".format(output_lable(pred_LR1[0]),))

Test score: 98.67 %


In [39]:
news = str(input())
model_testing(news)

When life in her country became too dangerous, Natalia* took her children and fled. She managed to cross into Mexico via the southern border, but once she arrived in the country, she had no means of providing for herself and her family.   Her story is similar to those of many other desperate people who head to Mexico, increasingly seen as a country of transit and asylum: In 2014, 2,100 people arrived in the country to request refugee status; five years later, in 2019, that had risen to more than 70,000.  The figures dropped in 2020, as travel restrictions imposed as a result of the COVID-19 pandemic slowed global migration but, between January and November 2021, the country received more than 123,000 asylum requests from people coming from Caribbean, Central American and South American countries (Haiti, Honduras, Cuba, El Salvador, Chile, Venezuela, Guatemala, Nicaragua, Brazil and Colombia).  Natalia and her children are now being supported by the UN refugee agency, (UNHCR), which has